In [71]:
import os
import gensim#from gensim.models import word2vec
import math
import random
import numpy as np
from numpy import linalg as la
import datetime
import time
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
random.seed?

In [4]:
random.sample(set([1,2,3,4,5]), 2)

[2, 5]

In [5]:
filter(lambda x: x % 2 == 1, [1,2,3,4,5])

[1, 3, 5]

In [13]:

def extract_data_from_file_and_generate_train_set_with_specified_M_and_R(filename, num_of_movies, num_of_rates, delimiter):
    train = None
    data = []
    movie_id = set()

    with open(filename , 'r') as f:
        for i, line in enumerate(f):
            userId, movieId, rating, timestamp = line.split(delimiter)
            #userId = int(userId)
            #movieId = int(movieId)
            rating = float(rating)
            timestamp = int(timestamp)
            
            movie_id.add(movieId)

            data.append((userId, movieId, rating, timestamp))

    # control number of movies
    movies_selected = random.sample(movie_id, num_of_movies)
    print 'num_of_movies:'
    print 'expected len: %d' % (num_of_movies)
    print 'in fact: %d' % len(movies_selected)
    
    # filter out history records whose movies are not selected
    data = filter(lambda x: x[1] in movies_selected, data) # Careful! Hard-coded number!
    
    # control number of rates
    data = random.sample(data, num_of_rates)
    print 'num_of_rates:'
    print 'expected len: %d' % (num_of_rates)
    print 'in fact: %d' % len(data)
    
    train = {}
    for i, t in enumerate(data):
        userId, movieId, rating, timestamp = t
        if userId not in train:
            train[userId] = []
        train[userId].append((movieId, rating, timestamp))
    
    return train


In [9]:
#data_filename, delimiter, data_set = os.path.sep.join(['ml-100k', 'u.data']), '\t', '100K'
data_filename, delimiter, data_set = os.path.sep.join(['ml-1m', 'ratings.dat']), '::', '1M'
#data_filename, delimiter, data_set = os.path.sep.join(['ml-10M100K', 'ratings.dat']), '::', '10M'


def get_lines(filename):
    cnt = 0
    with open(filename , 'r') as f:
        for cnt, line in enumerate(f):
            pass
    return cnt + 1

total_lines = get_lines(data_filename)
print 'total_lines:', total_lines

K = None
N = None


total_lines: 1000209


In [14]:
num_of_movies = 1000
num_of_rates = 10 * 10000
r = extract_data_from_file_and_generate_train_set_with_specified_M_and_R(data_filename, num_of_movies, num_of_rates, delimiter)
    

num_of_movies:
expected len: 1000
in fact: 1000
num_of_rates:
expected len: 100000
in fact: 100000


In [17]:
r.values()[0]

[('3060', 3.0, 959974376),
 ('1120', 3.0, 956875950),
 ('1747', 4.0, 959974407),
 ('296', 2.0, 956875720),
 ('3108', 4.0, 959974264),
 ('2124', 3.0, 959974504),
 ('1213', 3.0, 956875635),
 ('588', 3.0, 959974349),
 ('3253', 3.0, 959974328),
 ('1265', 4.0, 959974264),
 ('1569', 2.0, 959974560),
 ('3450', 3.0, 959974504)]

In [44]:
l = [[1,2], [3, 4]]
#print [x for x in l]
x = None
print [y for y in x for x in l]
print [y for x in l for y in x]

TypeError: 'NoneType' object is not iterable

In [34]:
[x for x in r.values()[:2]]

[[('3060', 3.0, 959974376),
  ('1120', 3.0, 956875950),
  ('1747', 4.0, 959974407),
  ('296', 2.0, 956875720),
  ('3108', 4.0, 959974264),
  ('2124', 3.0, 959974504),
  ('1213', 3.0, 956875635),
  ('588', 3.0, 959974349),
  ('3253', 3.0, 959974328),
  ('1265', 4.0, 959974264),
  ('1569', 2.0, 959974560),
  ('3450', 3.0, 959974504)],
 [('2829', 1.0, 957756760),
  ('317', 4.0, 956874389),
  ('1641', 3.0, 956873721),
  ('3053', 5.0, 957756905),
  ('2840', 4.0, 957756905),
  ('3253', 3.0, 956873744),
  ('3512', 5.0, 956873163)]]

In [45]:
#[y for y in x for x in r.values()[:2]] # WRONG!
[y for x in r.values()[:2] for y in x]

[('3060', 3.0, 959974376),
 ('1120', 3.0, 956875950),
 ('1747', 4.0, 959974407),
 ('296', 2.0, 956875720),
 ('3108', 4.0, 959974264),
 ('2124', 3.0, 959974504),
 ('1213', 3.0, 956875635),
 ('588', 3.0, 959974349),
 ('3253', 3.0, 959974328),
 ('1265', 4.0, 959974264),
 ('1569', 2.0, 959974560),
 ('3450', 3.0, 959974504),
 ('2829', 1.0, 957756760),
 ('317', 4.0, 956874389),
 ('1641', 3.0, 956873721),
 ('3053', 5.0, 957756905),
 ('2840', 4.0, 957756905),
 ('3253', 3.0, 956873744),
 ('3512', 5.0, 956873163)]

In [46]:
all_tuples = [y for x in r.values() for y in x]
distinct_items = set([x[0] for x in all_tuples])
print len(distinct_items)

963


In [47]:
list(distinct_items)[:10]

['3922', '3923', '1718', '3925', '3928', '1869', '1147', '3259', '1948', '669']

In [48]:
def convert_level_1_dict_level_2_list_of_size_3_tuples_to_list_of_list(data):
    return [map(lambda y: y[0], data[x]) for x in data]

In [68]:
m[[x for x in m.wv.vocab][0]]

array([ -1.12297408e-01,  -3.59947309e-02,   4.36375029e-02,
        -6.58282712e-02,  -5.41042387e-02,  -2.02573940e-01,
         5.55509552e-02,  -7.79519603e-02,   3.09215277e-01,
         2.49680951e-01,   3.67670804e-01,   3.21497858e-01,
         9.67275128e-02,  -2.09786549e-01,  -5.31960139e-03,
        -1.81605160e-01,  -1.28780171e-01,  -2.49864742e-01,
        -1.77400455e-01,   1.84372351e-01,   1.68940768e-01,
         2.54767090e-01,   8.53701606e-02,  -1.20013831e-02,
         1.35253340e-01,   7.03064725e-02,   2.16806889e-01,
        -2.46946931e-01,   1.01544999e-01,  -1.72019288e-01,
        -1.32090813e-02,   4.33598049e-02,  -7.59228989e-02,
        -3.28450859e-01,   6.33343682e-02,   1.94664538e-01,
         8.46228376e-02,   2.02172220e-01,   2.35088378e-01,
         2.34534308e-01,  -2.34135330e-01,   5.79590686e-02,
         3.68431094e-04,   5.25200069e-02,  -5.04669659e-02,
         5.31099504e-03,  -4.69869189e-02,  -2.35796690e-01,
         4.84678410e-02,

In [64]:
m = train_a_model(para)

2017-04-01 20:23:22,489 : INFO : collecting all words and their counts
2017-04-01 20:23:22,490 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-01 20:23:22,498 : INFO : collected 1678 word types from a corpus of 30000 raw words and 5156 sentences
2017-04-01 20:23:22,498 : INFO : Loading a fresh vocabulary
2017-04-01 20:23:22,503 : INFO : min_count=1 retains 1678 unique words (100% of original 1678, drops 0)
2017-04-01 20:23:22,503 : INFO : min_count=1 leaves 30000 word corpus (100% of original 30000, drops 0)
2017-04-01 20:23:22,507 : INFO : deleting the raw counts dictionary of 1678 items
2017-04-01 20:23:22,507 : INFO : sample=0.001 downsamples 46 most-common words
2017-04-01 20:23:22,508 : INFO : downsampling leaves estimated 29029 word corpus (96.8% of prior 30000)
2017-04-01 20:23:22,508 : INFO : estimated required memory for 1678 words and 100 dimensions: 2181400 bytes
2017-04-01 20:23:22,512 : INFO : resetting layer weights
2017-04-01 20:23:22,5

start training


2017-04-01 20:23:23,529 : INFO : PROGRESS: at 11.48% examples, 66440 words/s, in_qsize 4, out_qsize 4
2017-04-01 20:23:24,529 : INFO : PROGRESS: at 22.56% examples, 65330 words/s, in_qsize 4, out_qsize 3
2017-04-01 20:23:25,529 : INFO : PROGRESS: at 32.75% examples, 63332 words/s, in_qsize 5, out_qsize 1
2017-04-01 20:23:26,529 : INFO : PROGRESS: at 42.87% examples, 62167 words/s, in_qsize 1, out_qsize 1
2017-04-01 20:23:27,529 : INFO : PROGRESS: at 52.77% examples, 61222 words/s, in_qsize 0, out_qsize 1
2017-04-01 20:23:28,529 : INFO : PROGRESS: at 63.17% examples, 61099 words/s, in_qsize 0, out_qsize 1
2017-04-01 20:23:29,529 : INFO : PROGRESS: at 74.98% examples, 62171 words/s, in_qsize 2, out_qsize 0
2017-04-01 20:23:30,529 : INFO : PROGRESS: at 85.40% examples, 61950 words/s, in_qsize 5, out_qsize 2
2017-04-01 20:23:31,529 : INFO : PROGRESS: at 95.80% examples, 61760 words/s, in_qsize 1, out_qsize 4
2017-04-01 20:23:31,914 : INFO : worker thread finished; awaiting finish of 2 more

training finished
start modeling
modeling finished


In [69]:
def train_a_model(para):
    data = para['data']
    num_features = para['num_features']
    min_count = para['min_count']
    window = para['window']
    para_iter = para['iter']
    batch_words = para['batch_words']

    #list_of_list = convert_2_level_dict_to_list_of_list(data)
    list_of_list = convert_level_1_dict_level_2_list_of_size_3_tuples_to_list_of_list(data)
    #print 'list_of_list:', list_of_list

    print 'start training'
    model = gensim.models.Word2Vec(list_of_list, size=num_features, min_count=min_count, window=window, sg=0, iter=para_iter, batch_words=batch_words)
    print 'training finished'
    print 'start modeling'
    representation = [m[x] for x in m.wv.vocab]
    print 'modeling finished'
    return model, representation

In [89]:
r = generate_mimic_data_train_set_with_specified_M_and_R(1000, 100000)

In [90]:
r[0]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [76]:
def generate_mimic_data_train_set_with_specified_M_and_R(num_of_movies, num_of_rates):
    num_of_users = num_of_rates / num_of_movies
    train = {}
    history = [str(x) for x in range(num_of_movies)]
    for x in xrange(num_of_users):
        train[x] = history[:]
        
    return train

In [99]:
#train_set_size_list = 10 ** np.array(range(1, 4 + 1))
#train_set_size_list = 10**4 * np.array(range(1, 5 + 1))
#iter_list = [20, 30, 40]

#num_of_movies = None #1000
#num_of_rates = 10 * 10000

num_of_movies_list = [1000, 3000, 5000]
num_of_rates_list = 10**6 * np.array(range(1, 3 + 1))

metric = []

###

test_data_inner_ratio = 0.5  # insignificant in this notebook
s, mc, w = 100, 1, 1         # insignificant in this notebook
batch_words = 1          # insignificant in this notebook
para_iter = 20

seed = 2 

for __i, num_of_movies in enumerate(num_of_movies_list):
    for __j, num_of_rates in enumerate(num_of_rates_list):
        random.seed(seed)
        
        print 'start preparing training set.'
        #train = extract_data_from_file_and_generate_train_set_with_specified_M_and_R(data_filename, num_of_movies, num_of_rates, delimiter)
        train = generate_mimic_data_train_set_with_specified_M_and_R(num_of_movies, num_of_rates)
        print 'preparing training set done.'
        print 'len(train):', len(train)

        para = {'data': train, 
            'model_name': 'main_model',
            'num_features': s,
            'min_count': mc,
            'window': w,
            'iter': para_iter,
            'batch_words': batch_words,
        }
        ##
        print "currently, num_of_movies=%d, num_of_rates=%d." % (num_of_movies, num_of_rates)
        
        #starttime = datetime.datetime.now()             # start timing
        starttime = time.time()
        train_a_model(para)

        #endtime = datetime.datetime.now()               # stop timing
        endtime = time.time()               # stop timing
        
        #interval = (endtime - starttime).seconds
        interval = endtime - starttime
        print 'interval: %g' % (interval)
        
        metric.append((num_of_movies, num_of_rates, interval))
        
        break
    #break
print metric

2017-04-01 20:51:14,666 : INFO : collecting all words and their counts
2017-04-01 20:51:14,667 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-01 20:51:14,727 : INFO : collected 10 word types from a corpus of 1000000 raw words and 1000 sentences
2017-04-01 20:51:14,727 : INFO : Loading a fresh vocabulary
2017-04-01 20:51:14,728 : INFO : min_count=1 retains 10 unique words (100% of original 10, drops 0)
2017-04-01 20:51:14,731 : INFO : min_count=1 leaves 1000000 word corpus (100% of original 1000000, drops 0)
2017-04-01 20:51:14,732 : INFO : deleting the raw counts dictionary of 10 items
2017-04-01 20:51:14,732 : INFO : sample=0.001 downsamples 9 most-common words
2017-04-01 20:51:14,733 : INFO : downsampling leaves estimated 104820 word corpus (10.5% of prior 1000000)
2017-04-01 20:51:14,733 : INFO : estimated required memory for 10 words and 100 dimensions: 13000 bytes
2017-04-01 20:51:14,734 : INFO : resetting layer weights
2017-04-01 20:51:14,734 :

start preparing training set.
preparing training set done.
len(train): 1000
currently, num_of_movies=1000, num_of_rates=1000000.
start training


2017-04-01 20:51:15,736 : INFO : PROGRESS: at 21.37% examples, 447911 words/s, in_qsize 4, out_qsize 5
2017-04-01 20:51:16,736 : INFO : PROGRESS: at 45.38% examples, 475612 words/s, in_qsize 5, out_qsize 4
2017-04-01 20:51:17,737 : INFO : PROGRESS: at 64.99% examples, 454185 words/s, in_qsize 5, out_qsize 2
2017-04-01 20:51:18,737 : INFO : PROGRESS: at 86.53% examples, 453504 words/s, in_qsize 4, out_qsize 2
2017-04-01 20:51:19,429 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-01 20:51:19,430 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-01 20:51:19,431 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-01 20:51:19,431 : INFO : training on 20000000 raw words (2096260 effective words) took 4.7s, 446401 effective words/s
2017-04-01 20:51:19,555 : INFO : collecting all words and their counts
2017-04-01 20:51:19,556 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-01 20:51:19

training finished
start modeling
modeling finished
interval: 4.88822
start preparing training set.
preparing training set done.
len(train): 333
currently, num_of_movies=3000, num_of_rates=1000000.
start training


2017-04-01 20:51:20,627 : INFO : PROGRESS: at 23.54% examples, 426251 words/s, in_qsize 5, out_qsize 5
2017-04-01 20:51:21,627 : INFO : PROGRESS: at 46.98% examples, 424813 words/s, in_qsize 4, out_qsize 3
2017-04-01 20:51:22,628 : INFO : PROGRESS: at 70.47% examples, 424503 words/s, in_qsize 5, out_qsize 2
2017-04-01 20:51:23,629 : INFO : PROGRESS: at 94.29% examples, 425852 words/s, in_qsize 5, out_qsize 2
2017-04-01 20:51:23,875 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-01 20:51:23,876 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-01 20:51:23,877 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-01 20:51:23,883 : INFO : training on 19980000 raw words (1806652 effective words) took 4.3s, 424358 effective words/s
2017-04-01 20:51:24,004 : INFO : collecting all words and their counts
2017-04-01 20:51:24,005 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-01 20:51:24

training finished
start modeling
modeling finished
interval: 4.44458
start preparing training set.
preparing training set done.
len(train): 200
currently, num_of_movies=5000, num_of_rates=1000000.
start training


2017-04-01 20:51:25,078 : INFO : PROGRESS: at 24.93% examples, 460701 words/s, in_qsize 4, out_qsize 3
2017-04-01 20:51:26,079 : INFO : PROGRESS: at 49.35% examples, 456322 words/s, in_qsize 4, out_qsize 3
2017-04-01 20:51:27,080 : INFO : PROGRESS: at 74.12% examples, 456811 words/s, in_qsize 5, out_qsize 2
2017-04-01 20:51:28,079 : INFO : PROGRESS: at 98.72% examples, 456338 words/s, in_qsize 6, out_qsize 3
2017-04-01 20:51:28,121 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-01 20:51:28,122 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-01 20:51:28,122 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-01 20:51:28,124 : INFO : training on 20000000 raw words (1849474 effective words) took 4.0s, 456900 effective words/s


training finished
start modeling
modeling finished
interval: 4.23725
[(1000, 1000000, 4.88821816444397), (3000, 1000000, 4.444576025009155), (5000, 1000000, 4.237254858016968)]


In [98]:
for x in metric:
    print x

(1000, 100000, 0.4424738883972168)
(3000, 100000, 0.4133009910583496)
(5000, 100000, 0.40193986892700195)
